In [1]:
import pickle
import random
import numpy
import torch
import os

from Auxiliary_Functions.generate_observation import generate_observations, split_dataset
from Auxiliary_Functions.save_progress import save_progress
from Auxiliary_Functions.matrix_normalization import normalize_max_row_norm

from Denoising_Algorithms.Projectors.DnCNN import DnCNN
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR
from Denoising_Algorithms.DL_Training.loss_functions import IntermediateScaledLoss
from Denoising_Algorithms.DL_Training.training import train_main
from Denoising_Algorithms.Memory_Network.Memory_Net import MemoryNetwork
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR

In [2]:
current_trial = 0

torch.manual_seed(current_trial)
numpy.random.seed(current_trial)
random.seed(current_trial)
torch.cuda.manual_seed(current_trial)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/DCT_10.pkl', 'rb') as f:
    DCT_10 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/DCT_20.pkl', 'rb') as f:
    DCT_20 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/DCT_30.pkl', 'rb') as f:
    DCT_30 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/DCT_40.pkl', 'rb') as f:
    DCT_40 = pickle.load(f)

In [4]:
with open('ILSVRC2012_50x50_small.pkl', 'rb') as f:
    ILSVRC2012 = pickle.load(f)

# We only work with the first 25000 images
img_dataset = ILSVRC2012[:25000]

In [5]:
DCT_10_normalized = normalize_max_row_norm(DCT_10)
DCT_20_normalized = normalize_max_row_norm(DCT_20)
DCT_30_normalized = normalize_max_row_norm(DCT_30)
DCT_40_normalized = normalize_max_row_norm(DCT_40)

DCT_10_observations = generate_observations(img_dataset, DCT_10_normalized, 0.025)
DCT_20_observations = generate_observations(img_dataset, DCT_20_normalized, 0.025)
DCT_30_observations = generate_observations(img_dataset, DCT_30_normalized, 0.025)
DCT_40_observations = generate_observations(img_dataset, DCT_40_normalized, 0.025)

DCT_10_train, DCT_10_test = split_dataset(DCT_10_observations, train_ratio = 0.9, seed = 0)
DCT_20_train, DCT_20_test = split_dataset(DCT_20_observations, train_ratio = 0.9, seed = 0)
DCT_30_train, DCT_30_test = split_dataset(DCT_30_observations, train_ratio = 0.9, seed = 0)
DCT_40_train, DCT_40_test = split_dataset(DCT_40_observations, train_ratio = 0.9, seed = 0)

In [6]:
projector = DnCNN(depth = 3)

# Equal Weighting
loss_function = IntermediateScaledLoss(omega = 1.0)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# 5 Projections
num_projections = 5

# Folder Path for 5 Projections
proj5_path = 'Memory_Residual-DCT-Additive_Depth3-.025/5 Projections'

DCT_10_proj5_weights, DCT_10_proj5_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj5', display=True)

DCT_20_proj5_weights, DCT_20_proj5_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj5', display=True)

DCT_30_proj5_weights, DCT_30_proj5_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj5', display=True)

DCT_40_proj5_weights, DCT_40_proj5_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj5', display=True)

# Store Best Model Weights and History

save_progress(proj5_path, DCT_10_proj5_weights, DCT_10_proj5_hist, '10% Sampling')
save_progress(proj5_path, DCT_20_proj5_weights, DCT_20_proj5_hist, '20% Sampling')
save_progress(proj5_path, DCT_30_proj5_weights, DCT_30_proj5_hist, '30% Sampling')
save_progress(proj5_path, DCT_40_proj5_weights, DCT_40_proj5_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.060433595447076695
Average test loss: 0.005108636341161198
Epoch 2/300
Average training loss: 0.023992376729846002
Average test loss: 0.0046805844782955115
Epoch 3/300
Average training loss: 0.022902614570326274
Average test loss: 0.004517647260593043
Epoch 4/300
Average training loss: 0.022414540673295655
Average test loss: 0.004515911019303732
Epoch 5/300
Average training loss: 0.02213797199394968
Average test loss: 0.004542382898430029
Epoch 6/300
Average training loss: 0.02193121287557814
Average test loss: 0.004392952525367339
Epoch 7/300
Average training loss: 0.021783601290649837
Average test loss: 0.004368152441663875
Epoch 8/300
Average training loss: 0.02166708174182309
Average test loss: 0.004357085736261474
Epoch 9/300
Average training loss: 0.02155657383468416
Average test loss: 0.004320474741773473
Epoch 10/300
Average training loss: 0.021468567435940108
Average test loss: 0.00434366403416627
Ep

In [8]:
DCT_10_proj5_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj5_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj5_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj5_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj5_psnr = average_PSNR(DCT_10_proj5_model, DCT_10_proj5_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj5_psnr = average_PSNR(DCT_20_proj5_model, DCT_20_proj5_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj5_psnr = average_PSNR(DCT_30_proj5_model, DCT_30_proj5_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj5_psnr = average_PSNR(DCT_40_proj5_model, DCT_40_proj5_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj5_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj5_psnr, f)
with open(os.path.join(proj5_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj5_psnr, f)
with open(os.path.join(proj5_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj5_psnr, f)
with open(os.path.join(proj5_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj5_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 27.38
Average PSNR for Projection Layer 1 across 2500 images: 27.61
Average PSNR for Projection Layer 2 across 2500 images: 27.68
Average PSNR for Projection Layer 3 across 2500 images: 27.79
Average PSNR for Projection Layer 4 across 2500 images: 27.89
Average PSNR for Projection Layer 0 across 2500 images: 27.87
Average PSNR for Projection Layer 1 across 2500 images: 28.39
Average PSNR for Projection Layer 2 across 2500 images: 28.62
Average PSNR for Projection Layer 3 across 2500 images: 28.84
Average PSNR for Projection Layer 4 across 2500 images: 28.92
Average PSNR for Projection Layer 0 across 2500 images: 28.46
Average PSNR for Projection Layer 1 across 2500 images: 29.18
Average PSNR for Projection Layer 2 across 2500 images: 29.56
Average PSNR for Projection Layer 3 across 2500 images: 29.72
Average PSNR for Projection Layer 4 across 2500 images: 29.98
Average PSNR for Projection Layer 0 across 2500 images: 29.22
Average 

In [9]:
# 15 Projections
num_projections = 15

# Folder Path for 15 Projections
proj15_path = 'Memory_Residual-DCT-Additive_Depth3-.025/15 Projections'

DCT_10_proj15_weights, DCT_10_proj15_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj15', display=True)

DCT_20_proj15_weights, DCT_20_proj15_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj15', display=True)

DCT_30_proj15_weights, DCT_30_proj15_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj15', display=True)

DCT_40_proj15_weights, DCT_40_proj15_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj15', display=True)

# Store Best Model Weights and History

save_progress(proj15_path, DCT_10_proj15_weights, DCT_10_proj15_hist, '10% Sampling')
save_progress(proj15_path, DCT_20_proj15_weights, DCT_20_proj15_hist, '20% Sampling')
save_progress(proj15_path, DCT_30_proj15_weights, DCT_30_proj15_hist, '30% Sampling')
save_progress(proj15_path, DCT_40_proj15_weights, DCT_40_proj15_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.7472301466597451
Average test loss: 0.005642343551748329
Epoch 2/300
Average training loss: 0.1162312316497167
Average test loss: 0.005135427174882756
Epoch 3/300
Average training loss: 0.08467493845356835
Average test loss: 0.004760612784160508
Epoch 4/300
Average training loss: 0.07674572073088752
Average test loss: 0.004645885417444838
Epoch 5/300
Average training loss: 0.07325693128506343
Average test loss: 0.004562300014413065
Epoch 6/300
Average training loss: 0.07109279965029823
Average test loss: 0.00454732714055313
Epoch 7/300
Average training loss: 0.06957077228029569
Average test loss: 0.004453626770940092
Epoch 8/300
Average training loss: 0.06840863718920284
Average test loss: 0.004443738923304611
Epoch 9/300
Average training loss: 0.06744307521979014
Average test loss: 0.00448401766974065
Epoch 10/300
Average training loss: 0.0666972025235494
Average test loss: 0.004358811146683163
Epoch 11/300


In [10]:
DCT_10_proj15_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj15_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj15_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj15_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj15_psnr = average_PSNR(DCT_10_proj15_model, DCT_10_proj15_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj15_psnr = average_PSNR(DCT_20_proj15_model, DCT_20_proj15_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj15_psnr = average_PSNR(DCT_30_proj15_model, DCT_30_proj15_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj15_psnr = average_PSNR(DCT_40_proj15_model, DCT_40_proj15_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj15_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj15_psnr, f)
with open(os.path.join(proj15_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj15_psnr, f)
with open(os.path.join(proj15_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj15_psnr, f)
with open(os.path.join(proj15_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj15_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 27.22
Average PSNR for Projection Layer 1 across 2500 images: 27.16
Average PSNR for Projection Layer 2 across 2500 images: 27.43
Average PSNR for Projection Layer 3 across 2500 images: 27.60
Average PSNR for Projection Layer 4 across 2500 images: 27.64
Average PSNR for Projection Layer 5 across 2500 images: 27.74
Average PSNR for Projection Layer 6 across 2500 images: 27.76
Average PSNR for Projection Layer 7 across 2500 images: 27.79
Average PSNR for Projection Layer 8 across 2500 images: 27.81
Average PSNR for Projection Layer 9 across 2500 images: 27.88
Average PSNR for Projection Layer 10 across 2500 images: 27.87
Average PSNR for Projection Layer 11 across 2500 images: 27.91
Average PSNR for Projection Layer 12 across 2500 images: 27.90
Average PSNR for Projection Layer 13 across 2500 images: 27.90
Average PSNR for Projection Layer 14 across 2500 images: 27.93
Average PSNR for Projection Layer 0 across 2500 images: 27.52
Ave

In [11]:
# 30 Projections
num_projections = 30

# Folder Path for 30 Projections
proj30_path = 'Memory_Residual-DCT-Additive_Depth3-.025/30 Projections'

DCT_10_proj30_weights, DCT_10_proj30_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj30', display=True)

DCT_20_proj30_weights, DCT_20_proj30_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj30', display=True)

DCT_30_proj30_weights, DCT_30_proj30_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj30', display=True)

DCT_40_proj30_weights, DCT_40_proj30_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj30', display=True)

# Store Best Model Weights and History

save_progress(proj30_path, DCT_10_proj30_weights, DCT_10_proj30_hist, '10% Sampling')
save_progress(proj30_path, DCT_20_proj30_weights, DCT_20_proj30_hist, '20% Sampling')
save_progress(proj30_path, DCT_30_proj30_weights, DCT_30_proj30_hist, '30% Sampling')
save_progress(proj30_path, DCT_40_proj30_weights, DCT_40_proj30_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 6.204811606407166
Average test loss: 0.005923012406461769
Epoch 2/300
Average training loss: 1.167172887749142
Average test loss: 0.005072437677946355
Epoch 3/300
Average training loss: 0.6052171907954745
Average test loss: 0.004888723781953255
Epoch 4/300
Average training loss: 0.4142967560821109
Average training loss: 0.25884343394968246
Average test loss: 0.004634785628567139
Epoch 7/300
Average training loss: 0.21659329651461706
Average test loss: 0.004517973952823215
Epoch 8/300
Average training loss: 0.18776813639534845
Average test loss: 0.004504997529503372
Epoch 9/300
Average training loss: 0.17095572292804717
Average test loss: 0.004468836416800817
Epoch 10/300
Average training loss: 0.1611996421284146
Average test loss: 0.004459587679555019
Epoch 11/300
Average training loss: 0.1454706076118681
Average test loss: 0.004532197427004576
Epoch 14/300
Average training loss: 0.14230840020709568
Average tes

In [ ]:
DCT_10_proj30_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj30_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj30_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj30_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj30_psnr = average_PSNR(DCT_10_proj30_model, DCT_10_proj30_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj30_psnr = average_PSNR(DCT_20_proj30_model, DCT_20_proj30_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj30_psnr = average_PSNR(DCT_30_proj30_model, DCT_30_proj30_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj30_psnr = average_PSNR(DCT_40_proj30_model, DCT_40_proj30_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj30_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj30_psnr, f)
with open(os.path.join(proj30_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj30_psnr, f)
with open(os.path.join(proj30_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj30_psnr, f)
with open(os.path.join(proj30_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj30_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 27.07
Average PSNR for Projection Layer 1 across 2500 images: 27.30
Average PSNR for Projection Layer 2 across 2500 images: 27.30
Average PSNR for Projection Layer 3 across 2500 images: 27.44
Average PSNR for Projection Layer 4 across 2500 images: 27.54
Average PSNR for Projection Layer 5 across 2500 images: 27.68
Average PSNR for Projection Layer 6 across 2500 images: 27.64
Average PSNR for Projection Layer 7 across 2500 images: 27.61
Average PSNR for Projection Layer 8 across 2500 images: 27.71
Average PSNR for Projection Layer 9 across 2500 images: 27.74
Average PSNR for Projection Layer 10 across 2500 images: 27.78
Average PSNR for Projection Layer 11 across 2500 images: 27.63
Average PSNR for Projection Layer 12 across 2500 images: 27.72
Average PSNR for Projection Layer 13 across 2500 images: 27.81
Average PSNR for Projection Layer 14 across 2500 images: 27.68
Average PSNR for Projection Layer 15 across 2500 images: 27.72
Av